## TimingETFs.ipynb

Code for the Chicago Booth course on Quantitative Portfolio Management by Ralph S.J. Koijen and Federico Mainardi.

### Preliminaries

This code builds a sector-rotation strategy based on a momentum signal using ETFs and analyzes its performance.
- As always, the data can be found in the dropbox folder: https://www.dropbox.com/scl/fo/hrjspow2cpstfnoeqb23v/h?rlkey=j4fohf1s4e6fdy49p7bs71b7l&dl=0.
- Please download the file `ETFData.parquet`. 
- Make sure that `qpm.py` and `qpm_download.py` are in the same folder. These files are available on Canvas and in the dropbox folder. 

In [1]:
import qpm
import qpm_download
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from statsmodels.iolib.summary2 import summary_col

Here we specify whether we would like to download the data and the directory in which the data is stored.

In [3]:
import_data = True
_DATA_DIR = '../Data'

### Step 1. Load Data

We first load the data.

In [9]:
if import_data == True: 
    
    df = qpm_download.etfs('2003-01-01', '2023-07-31')
        
if import_data == False:
    
    # Load the data
    df = pd.read_parquet('%s/ETFdata.parquet' %(_DATA_DIR))

df.head()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Step 1. Import Daily Data
Done
Step 2. Import Monthly Data
Done
Step 3. Import Fama-French Factors


,date,ym,permno,retd,ticker,retM,mktrf,rf
0,2003-01-02,2003-01-01,88224.0,0.059287,IYZ,-0.047219,-0.0257,0.001
1,2003-01-03,2003-01-01,88224.0,-0.006934,IYZ,-0.047219,-0.0257,0.001
2,2003-01-06,2003-01-01,88224.0,0.082294,IYZ,-0.047219,-0.0257,0.001
3,2003-01-07,2003-01-01,88224.0,-0.021659,IYZ,-0.047219,-0.0257,0.001
4,2003-01-08,2003-01-01,88224.0,-0.033914,IYZ,-0.047219,-0.0257,0.001


We only need monthly excess returns for the strategy. 
- The first line constructs excess returns. 
- As the data set is daily, the second line converts it to monthly data. 

In [10]:
df['rete'] = df['retM'] - df['rf']
df = df[['ym', 'ticker', 'rete']].drop_duplicates()
df.head()

,ym,ticker,rete
0,2003-01-01,IYZ,-0.048219
21,2003-01-01,IYW,-0.012077
42,2003-01-01,IYF,-0.018739
63,2003-01-01,IYE,-0.021807
84,2003-01-01,IYK,-0.034948


#### Select Tickers and Signals

Here you can select the tickers you want. 
- For now, we will select five iShares sector ETFs as an example: Financials (IYF), Consumer staples (IYK), Technology (IYW), Telecommunications (IYZ), and Energy (IYE).

In [11]:
df = df[df['ticker'].isin(['IYF', 'IYK', 'IYW', 'IYZ', 'IYE'])].sort_values(['ticker', 'ym'])
df.head()

,ym,ticker,rete
63,2003-01-01,IYE,-0.021807
162,2003-02-01,IYE,0.020090
263,2003-03-01,IYE,0.009599
368,2003-04-01,IYE,-0.010589
473,2003-05-01,IYE,0.106871


We need a weighting scheme to combine these ETFs. We build a simple momentum strategy. 
- The first line creates a column of log returns.
- The second line computes the returns over the last 12 months.
- The last line skips the most recent month. 

In [ ]:
df['LNrete'] = np.log(1 + df['rete'])
df['mom'] = df.groupby('ticker')['LNrete'].transform(lambda x : x.rolling(12).sum())
df['mom'] = df['mom'] - df['LNrete']
df.tail(5)

Next, we compute the following weight
$$w_{it} = \frac{\exp(b*mom_{it})}{\sum_j \exp(b*mom_{jt})}. $$

where $b$ is a parameter that is set by the user and $mom_it$ is the value of the momentum signal computed above.
- By taking the exponential, we make sure that the weight remains positive for all ETFs. 
- By dividing by $\sum_j \exp(b*mom_{jt})$, we make sure that the weights add to 1 across all ETFs in that given month.
- The parameter $b$ controls how aggressively we tilt our portfolio. For $b\simeq 0$, we are close to equal weights; for large values of $b$, we pick the ETF with the most favorable momentum signal. This function is sometimes called the *softmax function* (https://en.wikipedia.org/wiki/Softmax_function).

The next block of the code computes the numerator, shifts the signal one period back, and drops observations for which the value of the numerator is not available.

In [ ]:
b_param = 10

df['mom'] = np.exp(b_param * df['mom'])
df['Lmom'] = df.groupby(['ticker'])['mom'].shift(1)
df = df.dropna(subset = ['Lmom'])

Next, we compute the denominator and the weights.

In [ ]:
df['TLmom'] = df.groupby('ym')['Lmom'].transform('sum')
df['weight'] = df['Lmom'] / df['TLmom']

We are now ready to plot our portfolio weights.

In [ ]:
df.sort_values(['ym','ticker'], ascending = [True, True], inplace = True)  

# Plot the portfolio weights 
df.set_index('ym')

plt.rcParams['font.size'] = 14
fig, ax = plt.subplots(1, 1, figsize = (12, 5))
df_pivot = df.pivot(index='ym', columns='ticker', values='weight')
df_pivot.plot(ax = ax)

plt.xlabel('Date')
plt.ylabel('Portfolio weight')
plt.legend()
plt.show()

### Step 2. Portfolio Construction

We now compute the portfolio returns (line 1-4). 
- As a benchmark, we will also compare the performance of this strategy to an equal-weighted strategy (line 5). 
- We also check (line 6) that the weights sum to 1.

In [ ]:
df['wgtd_rete'] = df['weight'] * df['rete']

df_rets = {}
df_rets['reteP'] = df.groupby('ym')['wgtd_rete'].sum()
df_rets['reteEW'] = df.groupby('ym')['rete'].mean()
df_rets['check_weights'] = df.groupby('ym')['weight'].sum()
df_rets = pd.DataFrame(df_rets)
print(df_rets.tail(5))

### Step 3. Portfolio Analytics

Regress the returns of the sector-rotation strategy on the equal-weighted average of the ETFs:

In [ ]:
results_ETF = smf.ols(formula='reteP ~ reteEW', data=df_rets).fit()
print(summary_col(results_ETF,stars=True))

Annualized alpha in %:

In [ ]:
print(results_ETF.params['Intercept'] * 1200)